# MLP with TensorFlow Keras

In [57]:
import tensorflow as tf # type: ignore
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json


In [58]:

import csv
from matplotlib import pyplot as plt
import numpy as np
import json
import pickle
import copy
from activ_func import Activation_Function, reluVect, sigmoidVect, softmax
from backprop_func import delta_linear_output, delta_relu_output, delta_sigmoid_output, delta_softmax_output, delta_linear_hidden, delta_relu_hidden, delta_sigmoid_hidden, delta_softmax_hidden

class Layer:
    def __init__(self, w: np.ndarray, activ_func: Activation_Function) -> None:
        if (w.ndim != 2):
            raise RuntimeError("Layer initialized with non 2-dimensional array")

        self.w = w
        self.n_inputs = w.shape[0]
        self.n_neurons = w.shape[1]
        self.activ_func = activ_func
        
    def getWeight(self):
        return self.w


class FFNN:
    def __init__(self, n_inputs: int, n_classes: int, learning_rate: float) -> None:
        self._n_inputs = n_inputs
        self._n_classes = n_classes

        self._targets: list[list[float]] = []
        self._input: list[list[float]] = []
        self._layers: list[Layer] = []

        self._current_output: np.ndarray = None

        self._learning_rate = learning_rate

    def get_output(self):
        return np.transpose(self._current_output).tolist()

    def addInput(self, newInput: list[float], target_output: list[float]):
        if self._n_inputs != len(newInput):
            raise RuntimeError("Added input with incorrect number of attributes")
        if self._n_classes != len(target_output):
            raise RuntimeError("Added target with incorrect number of classes")

        self._input.append(newInput)
        self._targets.append(target_output)

    def addLayer(self, newLayer: Layer):
        if len(self._input) == 0:
            raise RuntimeError("Input not defined before adding hidden layer")

        if (
            len(self._layers) == 0 and (len(self._input[0]) + 1) != newLayer.n_inputs
        ) or (len(self._layers) != 0 and (self._layers[-1].n_neurons + 1) != newLayer.n_inputs):
            raise RuntimeError(
                "Number of inputs in layer matrix does not match output from previous layer"
            )

        self._layers.append(newLayer)

    def feed_forward(self):
        for cur_input in self._input:
            layer_inputs: list[list[float]] = []
            layer_nets: list[list[float]] = []

            current = np.transpose(np.array([cur_input]))
            bias = np.array([[1.0]])

            for _, layer in enumerate(self._layers):
                current = np.concatenate((bias, current), axis=0)
                layer_inputs.append(current.copy().transpose().tolist())

                new_current = np.transpose(layer.w) @ current
                current = new_current
                layer_nets.append(current.copy().transpose().tolist())

                if layer.activ_func == Activation_Function.RELU:
                    current = reluVect(current)
                elif layer.activ_func == Activation_Function.SIGMOID:
                    current = sigmoidVect(current)
                elif layer.activ_func == Activation_Function.SOFTMAX:
                    current = softmax(current)

            self._current_output = current
            self.backwards_propagation(layer_inputs, layer_nets)

    def update_w(self, layer_idx: int, delta: np.ndarray, inputs: list[float]):
        input_mat = np.transpose(np.array(inputs))
        input_mat = np.tile(input_mat, (1, self._layers[layer_idx].n_neurons))
        self._layers[layer_idx].w += self._learning_rate * delta * input_mat

    def backwards_propagation(self, layer_inputs: list[list[float]], layer_nets: list[list[float]]):
        ds_delta: np.ndarray = None
        for idx, layer in enumerate(reversed(self._layers)):
            layer_idx = (-1-idx) % len(self._layers)
            nets = np.array(layer_nets[layer_idx]).transpose()

            if idx == 0:
                target = np.array(self._targets[layer_idx]).transpose()

                if layer.activ_func == Activation_Function.SOFTMAX:
                    ds_delta = delta_softmax_output(self._current_output, target, layer.n_inputs)
                elif layer.activ_func == Activation_Function.RELU:
                    ds_delta = delta_relu_output(self._current_output, target, nets, layer.n_inputs)
                elif layer.activ_func == Activation_Function.SIGMOID:
                    ds_delta = delta_sigmoid_output(self._current_output, target, layer.n_inputs)
                else:
                    ds_delta = delta_linear_output(self._current_output, target, layer.n_inputs)
            else:
                cur_delta = None
                layer_outputs = np.array(layer_inputs[layer_idx + 1][1:])

                if layer.activ_func == Activation_Function.SOFTMAX:
                    cur_delta = delta_softmax_hidden(layer_outputs, ds_delta, self._layers[layer_idx + 1].w, layer.n_inputs)
                elif layer.activ_func == Activation_Function.RELU:
                    cur_delta = delta_relu_hidden(nets, ds_delta, self._layers[layer_idx + 1].w, layer.n_inputs)
                elif layer.activ_func == Activation_Function.SIGMOID:
                    cur_delta = delta_sigmoid_hidden(layer_outputs, ds_delta, self._layers[layer_idx + 1].w, layer.n_inputs)
                else:
                    cur_delta = delta_linear_hidden(ds_delta, self._layers[layer_idx + 1].w, layer.n_inputs)

                self.update_w(layer_idx + 1, ds_delta, layer_inputs[layer_idx + 1])
                ds_delta = cur_delta
    def getLayers(self):
        return self._layers

In [59]:
import numpy as np

def d_relu(v: float, net: float):
    return 0 if net < 0 else v

d_relu_vect = np.vectorize(d_relu)

def d_softmax(o: float, net: float):
    return -o if net == 1.0 else 1-0

d_softmax_vect = np.vectorize(d_softmax)

"""
All variables output, target, nets are in the form of
single column matrix
"""

def delta_linear_output(output: np.ndarray, target: np.ndarray, n_inputs: int):
    output_mat = np.tile(np.transpose(output), reps=(n_inputs, 1))
    target_mat = np.tile(np.transpose(target), reps=(n_inputs, 1))
    return target_mat - output_mat

def delta_relu_output(output: np.ndarray, target: np.ndarray, nets: np.ndarray, n_inputs: int):
    output_mat = np.tile(np.transpose(output), reps=(n_inputs, 1))
    target_mat = np.tile(np.transpose(target), reps=(n_inputs, 1))
    nets_mat = np.tile(np.transpose(nets), reps=(n_inputs, 1))

    return d_relu_vect(target_mat - output_mat, nets_mat)

def delta_sigmoid_output(output: np.ndarray, target: np.ndarray, n_inputs: int):
    output_mat = np.tile(np.transpose(output), reps=(n_inputs, 1))
    target_mat = np.tile(np.transpose(target), reps=(n_inputs, 1))
    
    return (target_mat - output_mat) * output_mat * (1 - output_mat)

def delta_softmax_output(output: np.ndarray, target: np.ndarray, n_inputs: int):
    output_mat = np.tile(np.transpose(output), reps=(n_inputs, 1))
    target_mat = np.tile(np.transpose(target), reps=(n_inputs, 1))

    return d_softmax_vect(output_mat, target_mat)

def delta_linear_hidden(
    ds_delta: np.ndarray, ds_w: np.ndarray, n_inputs: int
):
    sigma_vect = np.array([[np.sum(ds_delta[row_idx] * ds_w[row_idx]) for row_idx in range(ds_w.shape[0])]])

    return np.tile(sigma_vect, reps=(n_inputs, 1))

def delta_relu_hidden(
    nets: np.ndarray, ds_delta: np.ndarray, ds_w: np.ndarray, n_inputs: int
):
    nets_vect = np.transpose(nets)
    # TODO check if this handles bias correctly
    sigma_vect = np.array([[np.sum(ds_delta[row_idx] * ds_w[row_idx]) for row_idx in range(ds_w.shape[0])]])

    return np.tile(d_relu_vect(sigma_vect, nets_vect), reps=(n_inputs, 1))


def delta_sigmoid_hidden(
    output: np.ndarray, ds_delta: np.ndarray, ds_w: np.ndarray, n_inputs: int
):
    output_vect = np.transpose(output)
    output_vect = output_vect * (1 - output_vect)

    sigma_vect = np.array([[np.sum(ds_delta[row_idx] * ds_w[row_idx]) for row_idx in range(ds_w.shape[0])]])

    return np.tile(output_vect * sigma_vect, reps=(n_inputs, 1))

def delta_softmax_hidden(
    output: np.ndarray, ds_delta: np.ndarray, ds_w: np.ndarray, n_inputs: int
):
    output_deltas = []
    o_list: list[float] = np.transpose(output).tolist()[0]

    ds_sums: list[float] = [np.sum(ds_delta[j] * ds_w[j]) for j in range(len(ds_delta))]

    for i, oi in enumerate(o_list):
        o_sum = 0
        for j, oj in enumerate(o_list):
            if (i == j):
                do_dnet = oi * (1 - oj)
            else:
                do_dnet = -oi * oj
            o_sum += do_dnet * ds_sums[j]
        output_deltas.append(o_sum)
    
    return np.tile(np.array([output_deltas]), reps=(n_inputs, 1))


In [64]:
filename = '../models/linear.json'
try:
    with open(filename, 'r') as file:
        json_data = json.load(file)
        
    input_data = np.array(json_data['case']['input'])
    target_data = np.array(json_data['case']['target'])
    initial_weights = [np.array(layer) for layer in json_data['case']['initial_weights']]
    n_attr = json_data["case"]["model"]["input_size"]
    n_classes = len(json_data["case"]["target"][0])
    learning_rate = json_data["case"]["learning_parameters"]["learning_rate"]
    
    fnaf = FFNN(n_attr, n_classes, learning_rate)

    input = json_data["case"]["input"]
    target = json_data["case"]["target"]
    
    # with open('../data/iris.csv', newline='') as f:
    #         reader = csv.reader(f)
    #         data = list(reader)[1:]

    
    for i in range(len(input)):
        fnaf.addInput(input[i],target[i])


    # for row in data[:50]:
    #     inputs = list(map(float, row[1:5]))

    #     if row[5] == "Iris-setosa":
    #         target = [1.0,0.0,0.0]
    #     elif row[5] == "Iris-versicolor":
    #         target = [0.0,1.0,0.0]
    #     else:
    #         target = [0.0,0.0,1.0]
        
    
    #     fnaf.addInput(inputs, target)
        
    for i, layer_info in enumerate(json_data["case"]["model"]['layers']):
        layer = Layer(np.array(json_data["case"]["initial_weights"][i]), Activation_Function(json_data["case"]["model"]['layers'][i]["activation_function"]))
        fnaf.addLayer(layer)

except FileNotFoundError:
    print(f"Error: The file '{filename}' was not found.")
except json.JSONDecodeError:
    print(f"Error: The file '{filename}' does not contain valid JSON.")
except KeyError as e:
    print(f"Error: Missing expected key {e} in the JSON structure.")
        


# fnaf = FFNN(n_attr, n_classes, learning_rate)

# data = []

# with open('../data/iris.csv', newline='') as f:
#         reader = csv.reader(f)
#         data = list(reader)[1:]

# for row in data[:50]:
#     inputs = list(map(float, row[1:5]))

#     if row[5] == "Iris-setosa":
#         target = [1.0,0.0,0.0]
#     elif row[5] == "Iris-versicolor":
#         target = [0.0,1.0,0.0]
#     else:
#         target = [0.0,0.0,1.0]
        
#     fnaf.addInput(inputs, target)

# w_hidden = np.random.uniform(-0.5, 0.5, size=(5, 4))
# w_out = np.random.uniform(-0.5, 0.5, size=(5, 3))

# layer_hidden = Layer(w_hidden, Activation_Function.LINEAR)
# layer_out = Layer(w_out, Activation_Function.SIGMOID)

# fnaf.addLayer(layer_hidden)
# fnaf.addLayer(layer_out)

fnaf.feed_forward()

print(fnaf.get_output())

layers = fnaf.getLayers()
for i in range(len(layers)):
    print(layers[i].w)

[[0.7, -1.1, 0.5]]
[[ 0.1  0.3  0.2]
 [ 0.4  0.2 -0.7]
 [ 0.1 -0.8  0.5]]


In [ ]:
data = pd.read_csv("../data/iris.csv")

In [ ]:
df_x = data.iloc[:, 1:5]
df_y = data.iloc[:, 5]

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Encode target labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df_y)

# Convert integers to one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(df_x, onehot_encoded, test_size=0.2, random_state=42)


In [ ]:
with open('../models/linear.json', 'r') as f:
    json_data = json.load(f)

# Extract data from JSON
input_data = np.array(json_data['case']['input'])
target_data = np.array(json_data['case']['target'])
initial_weights = [np.array(layer) for layer in json_data['case']['initial_weights']]

# Define the model architecture
model = tf.keras.Sequential()
for i, layer in enumerate(json_data['case']['model']['layers']):
    if i == 0:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], use_bias=True, activation=layer['activation_function'], input_shape=(json_data['case']['model']['input_size'],)))
    else:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], activation=layer['activation_function']))

# Set initial weights
for i, layer in enumerate(model.layers):
    weights = initial_weights[i][1:]
    biases = initial_weights[i][0]
    layer.set_weights([weights, biases])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=json_data['case']['learning_parameters']['learning_rate']),
              loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(input_data, target_data, epochs=json_data['case']['learning_parameters']['max_iteration'], batch_size=json_data['case']['learning_parameters']['batch_size'])

# Evaluate the model
final_weights = [layer.get_weights() for layer in model.layers]

# Check if final weights match the expected values
# Print final weights obtained from the model
print("Final weights:")
for i, (weights, biases) in enumerate(final_weights):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(weights)
    print("Biases:")
    print(biases)

# Print expected final weights from the JSON data
print("\nExpected final weights:")
for i, layer_weights in enumerate(json_data['expect']['final_weights']):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(layer_weights)

# Evaluate the model
test_loss = model.evaluate(input_data, target_data, verbose=0)
print("Test Loss:", test_loss)

# Calculate accuracy
predictions = model.predict(input_data, verbose=0)
accuracy = np.mean(np.equal(np.argmax(target_data, axis=1), np.argmax(predictions, axis=1)))
print("Accuracy:", accuracy)

1/1 [==============================] - 0s 295ms/step - loss: 0.2217 - accuracy: 1.0000
Final weights:
Layer 1:
Weights:
[[ 0.4999996   0.29999906 -0.7999995 ]
 [ 0.19999948 -0.70000094  0.40000072]]
Biases:
[0.19999921 0.39999843 0.10000105]

Expected final weights:
Layer 1:
Weights:
[[0.22, 0.36, 0.11], [0.64, 0.3, -0.89], [0.28, -0.7, 0.37]]
Test Loss: [0.023333027958869934, 1.0]
Accuracy: 1.0
